## 0. Setup
### 0.1. Imports

In [1]:
import bw2io
import bw2data
# type hints
from bw2io import SingleOutputEcospold2Importer

### 0.2. Variables

In [2]:
str_ecoinvent_version: str = '3.9.1'

### 0.2. file paths

In [3]:
path_dir_datasets_ecoinvent: str = '/Users/michaelweinold/data/data_raw/ecoinvent_' + str_ecoinvent_version + '_cutoff_ecoSpold02/datasets'

## 1. `Brightway` setup
### 1.1. list all projects and current project

In [4]:
bw2data.projects

Brightway2 projects manager with 6 objects:
	3.8
	3.9
	3.9.1
	default
	ecoinvent35
	ecoinvent38
Use `projects.report()` to get a report on all projects.

In [5]:
bw2data.projects.current

'default'

### 1.2. ensure no project is corrupted

if a project has been corrupted, simply delete it and re-populate it. \
🌐 compare [`bw2io` issue #118](https://github.com/brightway-lca/brightway2-data/issues/118)

In [7]:
def safe_delete_brightway_project(name: str) -> None:
    try:
        bw2data.projects.delete_project(
            name = name,
            delete_dir = True
        )
    except:
        pass

## 2. Import `Ecoinvent` Databases
### 2.1. `Ecoinvent 3.9.1`

"Which `bw2io` version to use?" - Compare the [Brightway mailing list](https://brightway.groups.io/g/development/message/154) and the [`bw2io` changelog](https://github.com/brightway-lca/brightway2-io/blob/main/CHANGES.md#085-2021-09-21): for Ecoinvent <3.8: use `0.8.7`

In [8]:
bw2io.__version__

(0, 8, 8)

In [9]:
safe_delete_brightway_project(str_ecoinvent_version)
bw2data.projects.set_current(str_ecoinvent_version)
bw2io.bw2setup()
importer: SingleOutputEcospold2Importer = bw2io.SingleOutputEcospold2Importer(
    dirpath = path_dir_datasets_ecoinvent,
    db_name = str_ecoinvent_version
)

Creating default biosphere

Applying strategy: normalize_units
Applying strategy: drop_unspecified_subcategories
Applying strategy: ensure_categories_are_tuples
Applied 3 strategies in 0.00 seconds
Title: Writing activities to SQLite3 database:
  Started: 04/01/2023 14:06:45
  Finished: 04/01/2023 14:06:45
  Total time elapsed: 00:00:00
  CPU %: 100.10
  Memory %: 0.51
Created database: biosphere3
Creating default LCIA methods

Wrote 762 LCIA methods with 227223 characterization factors
Creating core data migrations

Extracting XML data from 21238 datasets
Extracted 21238 datasets in 16.96 seconds


In [10]:
importer.apply_strategies()
importer.statistics()

Applying strategy: normalize_units
Applying strategy: update_ecoinvent_locations
Applying strategy: remove_zero_amount_coproducts
Applying strategy: remove_zero_amount_inputs_with_no_activity
Applying strategy: remove_unnamed_parameters
Applying strategy: es2_assign_only_product_with_amount_as_reference_product
Applying strategy: assign_single_product_as_activity
Applying strategy: create_composite_code
Applying strategy: drop_unspecified_subcategories
Applying strategy: fix_ecoinvent_flows_pre35
Applying strategy: drop_temporary_outdated_biosphere_flows
Applying strategy: link_biosphere_by_flow_uuid
Applying strategy: link_internal_technosphere_by_composite_code
Applying strategy: delete_exchanges_missing_activity
Applying strategy: delete_ghost_exchanges
Applying strategy: remove_uncertainty_from_negative_loss_exchanges
Applying strategy: fix_unreasonably_high_lognormal_uncertainties
Applying strategy: set_lognormal_loc_value
Applying strategy: convert_activity_parameters_to_list
App

(21238, 674593, 0)

## 3. Investigation

In [12]:
activity0: dict = importer.data[0]

In [13]:
keys = [
    'name',
    'reference product',
    'unit',
    'code',
    'type'
]
[activity0[key] for key in keys if key in activity0]

['market for cyanuric chloride',
 'cyanuric chloride',
 'kilogram',
 'f32ba57ebce8c08759275f740a88f370',
 'process']

In [19]:
ei38 = bw2data.Database('ei38')

In [43]:
biosphere = bw2data.Database("biosphere3")

In [52]:
biosphere.random()

'Boron carbide' (kilogram, None, ('air', 'low population density, long-term'))

In [56]:
biosphere.get('4eb24f81-0468-4b92-bae3-8b12dfc90438').as_dict()

{'categories': ('air', 'non-urban air or from high stacks'),
 'code': '4eb24f81-0468-4b92-bae3-8b12dfc90438',
 'CAS number': '014269-63-7',
 'name': 'Thorium-230',
 'database': 'biosphere3',
 'unit': 'kilo Becquerel',
 'type': 'emission'}

In [55]:
biosphere.get('fc1c42ce-a759-49fa-b987-f1ec5e503db1').as_dict()

{'categories': ('natural resource', 'in water'),
 'code': 'fc1c42ce-a759-49fa-b987-f1ec5e503db1',
 'CAS number': '007732-18-5',
 'name': 'Water, cooling, unspecified natural origin',
 'database': 'biosphere3',
 'unit': 'cubic meter',
 'type': 'natural resource'}

In [23]:
activity0['exchanges'][0]

{'flow': '76ab0018-e176-43e0-b42c-12e82b937fef',
 'type': 'technosphere',
 'name': 'hard coal ash',
 'classifications': {'CPC': ['39: Wastes or scraps']},
 'production volume': 2353465484.53608,
 'properties': {'wet mass': {'amount': 1.0, 'unit': 'kg'},
  'carbon content, non-fossil': {'amount': 0.0, 'unit': 'dimensionless'},
  'carbon content, fossil': {'amount': 0.0,
   'unit': 'dimensionless',
   'comment': 'Tab A.7 & A.8, ecoinvent v2.2 report 13_I'},
  'water in wet mass': {'amount': 0.0,
   'unit': 'kg',
   'comment': 'Tab A.7 & A.8, ecoinvent v2.2 report 13_I'},
  'dry mass': {'amount': 1.0, 'unit': 'kg'},
  'water content': {'amount': 0.0,
   'unit': 'dimensionless',
   'comment': 'water mass/dry mass'},
  'price': {'amount': 0.0,
   'unit': 'EUR2005',
   'comment': 'Waste products are assigned a price of zero as they are assumed to require further treatment.'},
  'carbon content': {'amount': 0.0,
   'unit': 'dimensionless',
   'comment': 'carbon content on a dry matter basis (